# Step 3: Normalisation

We have previously discussed the principles behind non-linear registration on this course. As a quick review, the reasons for normalising data to a standard space relate to issues of anatomical variability and the problems this poses for analysing groups of scans. In brief, the statistical analysis of fMRI data is performed in voxel-space. As such, it is necessary for all the images to be the same dimensions *and* for the same voxel coordinates to correspond to the same anatomical locations. Given how substantial anatomical variability can be, this will never be the case for raw fMRI data. As such, normalisation becomes a necessary step. However, given the complexity of the anatomical differences across subjects, simple affine registration approaches will not be suitable. Instead, we need to use *non-linear* methods to provide the best correspondance between the data and the standard space template.

## Direct Normalisation to a Template

The most basic form of spatial normalisation involves the direct registration of an image to an MNI template. This can be performed as either a *one-step* or *two-step* procedure.

In the past it was usual to try and register the functional data directly to an EPI MNI template. While this will work to align the coarse structures of the brain, because the resolution of fMRI data is so poor there is limited anatomical information that can be used to improve this alignment. As such, it is more typical these days to use a structural T1 scan of the same subject to estimate the normalisation to a high-resolution structural MNI template. The improved detail of the structural scan should result in a much better fit to the MNI template. The parameters of this transformation can then be applied to the functional data. So, we usually use a two-step registration approach, as opposed to a one-step approach, as shown in {numref}`norm-poldrack-fig` from Poldrack, Mumford and Nichols (2011).

```{figure} images/norm-poldrack.png
---
width: 800px
name: norm-poldrack-fig
---
Illustration of both one-step and two-step normalisation procedures.
```

## Unified segmentation

The approach to normalisation used by SPM is known as unified segmentation (Ashburner & Friston, 2005), where the generation of normalisation parameters is performed as part of the same algorithm used to segment a structural image into the different tissue classes. Although we do not need to understand too much about how this segmentation works, it can be useful in this context to have some understanding because it can help identify where issues may be when the normalisation appears to have failed.

### Gaussian Mixture Models for Segmentation

In terms of segmenting an image, given that we can see the different tissue types fairly easily in a standard T1-weighted structural scan, it stands to reason that the voxel intensities should provide some information about the tissue classes. For instance, white matter is generally brighter than grey matter and CSF tends to be the darkest. We can get a sense of this by plotting the probabilities of intensity values for different manually-defined tissue types, as shown in {numref}`seg-tissues-fig`.

```{figure} images/seg-tissues.png
---
width: 600px
name: seg-tissues-fig
---
Example of the different intensity distribution for different tissue classes from a T1-weight anatomical image.
```

What is notable here is both the separation and overlap between the different probability distributions for each tissue. If the distributions were entirely separate then we could easily extract the values we needed. For instance, if this image only contained white matter (green curve) and CSF (red curve), then we could just classify any voxel with a value < 350 as CSF and any voxel with a value > 350 as white matter. Unfortunately, the degree of overlap between these distribution causes problems in terms of deciding whether a voxel is one tissue type or another. Leaving this issue to one side for now, we can model the probability distributions for different tissues in an image by using what is known as a Gaussian mixture model. This is illustrated in {numref}`gauss-mix-fig`, where the yellow bars represent the histogram of image intensities, the black line represents the overall shape of the distribution and the blue lines show how this distribution can be approximated by mixing together several Gaussian distributions (shown in blue), with different means and variances. This is very similar to the idea of recreating a signal using a linear combination of sines and cosines. So this could be considered a Gaussian basis set for the image intensities.

```{figure} images/gauss-mix.png
---
width: 600px
name: gauss-mix-fig
---
Illustration of a Gaussian mixture model.
```

After estimating the shapes of all the Gaussian distributions in blue, we can assess the probability that a voxel belongs to a tissue class by calculating the probability of its intensity value for each of these distributions. Whichever distribution gives the highest probability represents the most likely tissue class. However, as alluded to earlier, one of the issues with this is that there is a degree of overlap between these distributions. This means that certain intensity values will have similar probabilities for multiple tissue types. This speaks to a more general issue known as the partial volume effect, where a voxel can actually contain several tissue types, given that voxels can straddle tissue boundaries. As such, some additional information is useful in order to make a decision. This additional information comes in the form of tissue probability maps (TPMs), which are images that contain probabilities that indicate how likely each voxel in the brain is to be a particular tissue type. This therefore compliments the intensity information from the mixture-model with spatial information from the TPMs. The way SPM combines this information is based on a complicated Bayesian model where the TPMs are used as priors. The actual mechanics of this are not so important, but the principle of using both intensity and spatial information to decide what tissue type a voxel contains is hopefully fairly intuitive.

### Tissue Probability Maps (TPMs)

In terms of normalisation, the important element here are the TPMs. These are illustrated in {numref}`tpm-fig` for grey matter, white matter, CSF and "other". The TPMs in SPM12 are based on images from the ICBM Probabilistic Atlas, which is built from scans of 452 young healthy adults. The main point is that the TPMs in SPM are in MNI space and so in order to use them to make decisions about each subject's structural data they need warping into subject-space. Because of this, a non-linear deformation field that goes from MNI-space to subject-space is needed for the segmentation algorithm to work. This deformation field is therefore a byproduct of segmentation, that is constructed and estimated using the approaches we have discussed previously on the course. Importantly, inverting this field therefore provides us with the parameters we need to normalise our data.

```{figure} images/tpm.png
---
width: 400px
name: tpm-fig
---
Example of the tissue probability maps (TPMs) for grey matter, white matter, CSF and air.
```

### Anatomical and Functional Registration

Now remember, we are not trying to normalise the structural data. Instead, we want to use the normalisation parameters to normalise our functional volumes. In order for this to be possible, we have to make sure that the structural and functional scans are accurately registered. If this is not the case, then the deformation field will warp the wrong parts of the image and our transformation will be garbage. So the first step in unified segmentation is usually to register the structural and functional images together. Once we are happy with their alignment, we can use the structural image to estimate the transformation and then apply it to the functional images. The unified segmentation approach is therefore a two-step normalisation procedure.

## Normalising functional data in SPM

### Using the Graphical Batch System

The video below will demonstrate how to normalise the realigned and slice-time corrected functional data, using SPM12.

<div style="max-width: 1280px"><div style="position: relative; padding-bottom: 56.25%; height: 0; overflow: hidden;"><iframe src="https://livemanchesterac.sharepoint.com/sites/UOM-FBMH-MSc-NCCN/_layouts/15/embed.aspx?UniqueId=a33e96b4-97a3-48af-9a47-ef9c17b11c2c&embed=%7B%22hvm%22%3Atrue%2C%22ust%22%3Atrue%7D&referrer=StreamWebApp&referrerScenario=EmbedDialog.Create" width="1280" height="720" frameborder="0" scrolling="no" allowfullscreen title="5.Normalisation.mov" style="border:none; position: absolute; top: 0; left: 0; right: 0; bottom: 0; height: 100%; max-width: 100%;"></iframe></div></div>

### Using MATLAB Code

Unlike the previous preprocessing steps, there are no low-level functions in SPM for performing the unified segmentation approach to normalisation. As such, if we want to script this step we have to use the code generated by the batch system, which we will be discussing later in the course. For the time being, the older non-segmentation approach to normalisation is still available in the form of the `spm_normalise` and `spm_write_sn` functions, as demonstrated below. The corresponding batch modules can be found under `SPM > Tools > Old Normalise`. We can also see the use of the `spm_coreg` function for registering the anatomical and mean functional images.

In [3]:
% Launch SPM
spm('defaults','FMRI');
spm_figure('GetWin','Graphics');                   % Open graphics window for plots
addpath(fullfile(spm('dir'),'toolbox','OldNorm')); % Add the Old Normalise functions

% Load data
func  = spm_vol('arIA_preproc_func.nii');           % Motion and slice timing corrected data
mfunc = spm_vol('meanIA_preproc_func.nii');         % Mean functional (from realignment)
anat  = spm_vol('IA_preproc_anat.nii');             % Anatomical scan

% Coregister anat -> mean
spm_coreg();

% Normalise (estimate)
spm_normalise();
 
% Normalise (write)
spm_write_sn();


SPM12: spm_coreg (v7320)                           16:29:55 - 01/11/2023


Operands to the logical AND (&&) and OR (||) operators must be convertible to logical scalar values. Use the ANY or ALL functions to reduce operands to logical scalar values.

Error in spm_coreg>loaduint8 (line 229)
if size(V.pinfo,2)==1 && V.pinfo(1) == 2

Error in spm_coreg (line 127)
    VG.uint8 = loaduint8(VG);